# ISOLATION FOREST FOR ANOMALY DETECTION 


Algoritmos de Isolation Forest para detecção de Anolamia na Folha de Pagamento
Importação do Modulo de Pré-processamento

In [3]:
import sys
sys.path.append("C:/Users/joaoc/Documents/MT/pagamento_servidores/src")
import process_servants as ps
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import pandas as pd
from janitor import clean_names

## Função para pre-processamento dos dados
Utiliza-se funções importadas do arquivo process_servants.py, no qual foram pré-definidas algumas funções para o pré-processamento dos dados

In [5]:
def process_registration_data_rendim(file_path, reference_date_str):
    """Process registration data from Excel file"""
    
    df = (
        pd.read_excel(file_path)
        .pipe(clean_names)
        .pipe(ps.convert_rendim_to_numeric)
        .pipe(ps.process_month_column)
        )    

    #Selecionar apesas as colunas necessárias para o modelo
    df_rendim = df[['cpf_servidor', 'mes', 'rendim']]
    #Se tiver valores com o CPF duplicado para o mesmo mês, somar os valores
    df_rendim = df_rendim.groupby(['cpf_servidor', 'mes'])['rendim'].sum().reset_index()

    return df_rendim

## Função para Leitura dos Dados

In [7]:
def processed_data(brute_data_file_path, reference_date, file_name): 

    registration_df = process_registration_data_rendim(brute_data_file_path, reference_date)
    
    processed_path  = "C:/Users/joaoc/Documents/MT/pagamento_servidores/data/processed/" + file_name + ".xlsx"
    print("Processed data shapes:")
    print(f"Registration data: {registration_df.shape}")
    
    #registration_df.to_excel(processed_path, index=False)
    
    return registration_df

Importação da Base de Dados

In [9]:
reference_date = '2022-04-01'
brute_file_path = 'C:/Users/joaoc/Documents/MT/pagamento_servidores/data/raw/servidores_mes.xlsx'

Limpeza e Pre-processamento dos dados para tentar identificar o caso a seguir como um Anomaly Detection
![image.png](attachment:1cf54f27-e734-4093-a92f-766002dc7ed0.png)

In [11]:
registration_df = processed_data(brute_file_path, reference_date, "dados_12_meses_cadastro_e_std")

Processed data shapes:
Registration data: (41598, 3)


## Isolation Forest

In [13]:
resultados = []
for id in registration_df['cpf_servidor'].unique():
    # Filtrar dados do funcionário
    dados = registration_df[registration_df['cpf_servidor'] == id].sort_values('mes')
    
    if len(dados) < 12:  # Ignorar funcionários com menos de 12 meses de histórico
        continue
    
    serie = dados.set_index('mes')['rendim']
    contamination = 1/len(serie)
    
    iso_forest = IsolationForest(
        contamination = contamination,
        random_state = 42
    )

    x_iso_forest = serie.values.reshape(-1, 1)

    iso_forest.fit(x_iso_forest)

    anomalias = iso_forest.predict(x_iso_forest)

    dados['anomalia'] = anomalias

    for index, row in dados.iterrows():
        if row['anomalia'] == -1:
            resultados.append({
                'cpf_servidor': id,  # Supondo que a coluna com o ID se chama 'id'
                'mes': row['mes']  # Supondo que a coluna com o mês se chama 'mes'
            })
        


In [14]:
resultados

[{'cpf_servidor': 7547118623, 'mes': Timestamp('2022-06-01 00:00:00')},
 {'cpf_servidor': 7587804943, 'mes': Timestamp('2021-01-01 00:00:00')},
 {'cpf_servidor': 7651516184, 'mes': Timestamp('2020-09-01 00:00:00')},
 {'cpf_servidor': 7658751103, 'mes': Timestamp('2023-05-01 00:00:00')},
 {'cpf_servidor': 7662728188, 'mes': Timestamp('2023-05-01 00:00:00')},
 {'cpf_servidor': 7663707125, 'mes': Timestamp('2020-01-01 00:00:00')},
 {'cpf_servidor': 7664710472, 'mes': Timestamp('2023-11-01 00:00:00')},
 {'cpf_servidor': 7671550175, 'mes': Timestamp('2021-03-01 00:00:00')},
 {'cpf_servidor': 7681360166, 'mes': Timestamp('2023-11-01 00:00:00')},
 {'cpf_servidor': 7694184493, 'mes': Timestamp('2024-02-01 00:00:00')},
 {'cpf_servidor': 7694896647, 'mes': Timestamp('2022-11-01 00:00:00')},
 {'cpf_servidor': 7702843101, 'mes': Timestamp('2020-01-01 00:00:00')},
 {'cpf_servidor': 7703674109, 'mes': Timestamp('2020-11-01 00:00:00')},
 {'cpf_servidor': 7707249143, 'mes': Timestamp('2020-01-01 00:00

In [15]:
anomalias

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [16]:
dados['anomalia'] = anomalias

In [17]:
dados

,cpf_servidor,mes,rendim,anomalia
41552,35042770763,2020-01-01,0.00,1
41553,35042770763,2020-02-01,0.00,1
41554,35042770763,2020-03-01,0.00,1
41555,35042770763,2020-04-01,0.00,1
41556,35042770763,2020-05-01,0.00,1
41557,35042770763,2020-06-01,0.00,1
41558,35042770763,2020-07-01,0.00,1
41559,35042770763,2020-08-01,0.00,1
41560,35042770763,2020-09-01,0.00,1
41561,35042770763,2020-10-01,0.00,1
